In [9]:
import torch
# 各個元件 conv2d,dropout,....
import torch.nn as nn
# 各種function: mapooling,ReLu,Loss,.....
import torch.nn.functional as F
# 各種最佳化(gradient descent)方式: adam, SGD,....
import torch.optim as optm

In [2]:
'''
建立neural network:
1. extend nn.Module
2. override __init__
   initialize your neural network components at here. ex: conv2d, dropout,......
3. override forward
   to define you network architecture at here
'''
class minst_nn(nn.Module):
    def __init__(self):
        super(minst_nn, self).__init__()
        
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.bn2 = nn.BatchNorm2d(64)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
  
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        
        
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        
        return output
        
        

In [4]:
from torchvision import datasets, transforms
data_train = datasets.MNIST(root = "./data/",
                            transform=transforms,
                            train = True,
                            download = True)

Files already downloaded


## 資料讀取
#### torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False,  sampler=None, batch_sampler=None, num_workers=0,     collate_fn=None, pin_memory=False, drop_last=False, timeout=0, worker_init_fn=None, multiprocessing_context=None)
shuffle: 隨機排序              
### Pytorch基本圖像操作
#### torchvision.transforms
在使用torchvision.transforms時，我們通常使用transforms.Compose將transforms组合在一起。 <br>
transform = transforms.Compose([transforms list])
1. PIL.image,numpy.ndarray 和 pytorch tensor 轉換 <br>
   a. transforms.ToTensor() 轉成(c,h,w)範圍[0,1]<br>
   b. transforms.ToPILImage() 轉成(c,h,w)範圍[0,1]<br>
   c. transfroms.numpy() <br>
2. normalization <br>
   a. transforms.Normalize((mean),(std)) : 將範圍轉成[-1,1]
3. 其他圖像前處理(crap...等)

In [5]:
'''
計算mean以及std，供transforms.Normalize使用
這邊dataset以datasets.MNIST為例
'''
# import numpy as np
# def get_mean_std(dataset):
#     train_loader = torch.utils.data.DataLoader(
#             datasets.MNIST('./data', train=True, download=True,
#                            transform=transforms.Compose([
#                                transforms.ToTensor()
#                            ])),batch_size=60000, shuffle=True)
#     train = iter(train_loader).next()[0]
#     mean = np.mean(train.numpy(), axis=(0,1,2,3))
#     std = np.std(train.numpy(), axis=(0,1,2,3))
#     return mean, std

'\n計算mean以及std，供transforms.Normalize使用\n這邊dataset以datasets.MNIST為例\n'

In [6]:
def load_data(train_batch_size, test_batch_size):
    train_loader = torch.utils.data.DataLoader(
            datasets.MNIST('./data', train=True, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=train_batch_size, shuffle=True)#, **kwargs)
    test_loader = torch.utils.data.DataLoader(
            datasets.MNIST('./data', train=False, download=True,
                           transform=transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))
                           ])),
            batch_size=train_batch_size, shuffle=True)#, **kwargs)
    return (train_loader, test_loader)

### Train Model
#### Design the train & evaluate fucntion

##### Train

* ###### tqdm(進度條)
  tqdm(train_set, leave=True, total=len(train_set))

* ###### train
  1. 將model設定成train模式
  2. 將data, target放入device
  3. [(重要)更新參數前一定要將之前的grad設定成0](https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
     
  4. 將data丟入model
  5. 計算loss
  6. backpropogation
  7. 更新參數optimizer.step()

* ###### evaluation
  1. 將model設定成evaluation模式 <br>
     eval()时，框架會自動把BN和DropOut固定住，不會取平均，而是用訓練好的值，不然的话，一旦test的batch_size过小，很容易就會被BN層導致  生成圖片顏色失真！
  2. evaluation不需要計算gradien因此需要設定成torch.no_gradt模式
  3. 將data, target放入device
  4. 將data丟入model
  5. 計算loss
  6. 將out轉成target的格式 <br>
     output.argmax(dim=1, keepdim=True)
  7. 計算準確率 <br>
     correct += pred.eq(target.view_as(pred)).sum().item()

In [6]:
from tqdm import tqdm

In [28]:
# def train(arg, train_set, model, optimizer, epoch):
def train(train_set, model, optimizer, epoch, num_epochs, device):
    model.train()
    idx = 0
    with tqdm(train_set, leave=True, total=len(train_set)) as t:
        t.set_description('Epoch %d/%d' %(epoch, num_epoch))
        for data, target in t:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            out = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            idx += 1
            if idx % len(train_set) != 0:
                t.set_description(train_loss=loss.item())
            else:
                val_loss, accuracy = evaluate(globals.test_set, model)
                t.set_description(train_loss=loss.item(), val_loss=val_loss, acc=accuracy)
            

def evaluate(test_set, model, device):
    model.eval()
    loss = 0
    correct = 0
    for data, target in test_set:
        with torch.no_grad():
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss += F.nll_loss(output, target)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    accuracy = 100. * correct / len(test_set.dataset)
    return loss, accuracy
            

### Train Model

#### training
1. 將model丟入device(GPU or CPU)
2. 設定optimizer & 調整learning rate的方式(optional)


In [8]:
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same
#GPU版本的torch要加上.to(device)，不然會出現cpu版本跟gpu版本tensor 導致error
model = minst_nn().to(device)
optimizer = optm.Adam(model.parameters())
schedule = StepLR(optimizer, step_size=3, gamma=0.1)

In [10]:
(train_set, test_set) = load_data(64, 64)

Files already downloaded
Files already downloaded


In [ ]:
num_epochs = 10
for epoch in range(1, num_epochs+1):
    train(train_set, model, optimizer, epoch, num_epochs, device)
    

In [18]:
'''
jupyter notebook實驗用寫法
'''
# num_epochs = 10
# for epoch in range(1, num_epochs+1):
#     model.train()
#     with tqdm(train, leave=True, total=len(train)) as t:
#         t.set_description('Epoch %4d/%4d' %(epoch, num_epochs))
#         idx = 0
#         for data, target in t:
#             data, target = data.to(device), target.to(device)
#             optimizer.zero_grad()
#             output = model(data)
#             loss = F.nll_loss(output, target)
#             loss.backward()
#             optimizer.step()
            
#             idx += 1
#             if idx % len(train) != 0:
#                 t.set_postfix(train_loss=loss.item())
#             else:
#                 model.eval()
#                 val_loss = 0
#                 correct = 0
#                 with torch.no_grad():
#                     for data, target in test:
#                         data, target = data.to(device), target.to(device)
#                         output = model(data)
#                         val_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
#                         pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
#                         correct += pred.eq(target.view_as(pred)).sum().item()

#                 val_loss /= len(test.dataset)
#                 t.set_postfix(train_loss=loss.item(), val_loss=val_loss, acc= 100. * correct/len(test.dataset))


        
        

KeyboardInterrupt: 

#### model架構可視化
* ##### 直接print
* ##### 使用torchsummary套件(keras格式)

In [12]:
print(model)

minst_nn(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [13]:
from torchsummary import summary
summary(model,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
       BatchNorm2d-2           [-1, 32, 26, 26]              64
            Conv2d-3           [-1, 64, 24, 24]          18,496
       BatchNorm2d-4           [-1, 64, 24, 24]             128
         Dropout2d-5           [-1, 64, 12, 12]               0
            Linear-6                  [-1, 128]       1,179,776
         Dropout2d-7                  [-1, 128]               0
            Linear-8                   [-1, 10]           1,290
Total params: 1,200,074
Trainable params: 1,200,074
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.96
Params size (MB): 4.58
Estimated Total Size (MB): 5.55
----------------------------------------------------------------


### load custom dataset 
1. 要創建一個class繼承 torch.utils.data.Dataset <br>
2. override __init__, __len__, __getitem__ <br>
3. 傳入Dataloader

In [14]:
from torch.utils.data import Dataset
from glob import glob
import os, io
import pandas as pd
class tree_leaf_dataset(Dataset):
    def __init__(self, root_dir, train=True):
        self.root_dir = root_dir
        self.train = train
        if train: 
            dir_type = 'train'
        else:
            dir_type = 'test'
        paths = []
        labels = []
        label = 0
        folders = glob('r',root_dir + '/' + dir_type + '/image/*')
        for f in folders:
            if os.path.isdir(f):
                paths += glob(f + '/*.[Jj][Pp][Gg]')
                labels += [label] * len(paths)
        image_df = pd.DataFrame({
            'path': paths,
            'label': labels
        })
        self.image_df = image_df
    def __len__(self):
        return len(self.image_df)
    def __getitem__(self, idx):
        if torch.is_tensor():
            idx = idx.tolist()
        image_path = self.image_df.iloc[idx, 0]
        image = io.imread(image_path)
        label = self.image_df.iloc[idx, 1]
        sample = {'image': image, 'label': label}
        
        return sample

### command line 
#### argparse.ArgumentParser
add_argument常用參數 <br>
   1. '-xxx' & 'xxx' : 有-帶表示optional argument，沒有則是positional argument <br>
   2. type : 型別 <br>
   3. default <br>
   4. dest : 指定寫code的時候call該參數的參數名 <br>
   5. metavar: 代表help中顯示的參數名,通常記入完整的參數名
   6. help: help中的幫助訊息


In [7]:
# import argparse

# parse = argparse.ArgumentParser()
# 試寫
# parse.add_argument('-batch_size', type=int, default='64', dest='train_batch', metavar='train batch size',\
#                    help='input batch size for training (default: 64)')
# parse.add_argument('-test_batch_size', type=int, default='64', dest='test_batch',metavar='test_batch_size',\
#                   help='input batch size for training (default: 64)'))

#實務上做法是將參數寫在config,命令列只傳config檔路徑
# parse.add_argument('-c', type=str, dest= 'config_path', metavar='config file path')